# Automating Abstraction Error

After exploring the idea of abstraction [Rischel2020] and transformation [Rubenstein2017] in the first three notebooks, we now focus on the more practical problem of how, given two SCMs and an abstraction between them, we can compute the abstraction error between them.

In this notebook we will develop code to compute exactly the abstraction error as defined in [Rischel2020] and in the previous notebook. To achieve our goal, we will write functions that will be dependant on the metric we want to adopt and on the set of nodes over which we will evaluate the abstraction error. We will present a series of implementation progressively considering richer sets over which to compute the abstraction error.

This notebook was developed in order to offer a practical implementation of the ideas presented in the framework introduced in [Rischel2020], and to lay stonger foundations to further work with the idea of abstraction of causal models. The notebook is structured as follows: 
- Setup of standard and custom libraries (Section 2)
- Review of the definition of abstraction and notation (Section 3)
- Automated computation of abstraction error given pairs of variables in M1 connected by a directed path in M1 (Section 4)
- Automated computation of abstraction error given all pairs of variables in M1 (Section 5)
- Automated computation of abstraction error given pairs of variables in M1 connected by a directed path in M0 or M1 (Section 6)
- Automated computation of abstraction error given sets of variables in M1 connected by a directed path in M0 or M1 (Section 7)
- Integration of the code for the automated computation in the *Abstraction* class (Section 8)

DISCLAIMER 1: the notebook refers to ideas from *causality* and *category theory* for which only a quick definition is offered. Useful references for causality are [Pearl2009,Peters2017], while for category theory are [Spivak2014,Fong2018].

DISCLAIMER 2: mistakes are in all likelihood due to misunderstandings by the notebook author in reading [Rischel2020]. Feedback very welcome! :)

# Setup

## Standard libraries
First of all we import standard libraries.

In [1]:
import numpy as np
import networkx as nx
import itertools
from scipy.spatial import distance

For reproducibility, and for discussing our results in this notebook, we set a random seed to $1985$.

In [2]:
np.random.seed(1985)

## SCM implemenation

So far we have used a custom class to define and run our SCMs; we defined this class in the previous notebooks and we exported it into *src/models.py*. We will now switch to a standard library to implement Bayesian networks (BN): [pgmpy](https://pgmpy.org/index.html). *pgmpy* will allow us to define our SCMs as BNs and use existing functions to perform inferences on our network.

To use *pgmpy* we import the relevant libraries:

In [3]:
from pgmpy.models import BayesianNetwork as BN
from pgmpy.factors.discrete import TabularCPD as cpd
from pgmpy.inference import VariableElimination

## Abstraction definition

To be compatible with our new objects, we need to slightly revise the *Abstraction* class. Before, our class expected to work with custom *FinStochSCM* objects; now it will work with *BayesianNetwork* objects.

Importantly, we had to redefine the components of an abstraction:
- *R* is now a list of strings, each one denoting a variable in the base model $\mathcal{M}$/$\mathtt{M0}$ deemed relevant;
- *a* is now a dictionary indexed by the relevant variables in *R*; it specifies on which variable in the abstracted model $\mathcal{M'}$/$\mathtt{M1}$ the relevant variables are mapped.
- *alphas* is now dictionary indexed by variables in the abstracted model $\mathcal{M'}$/$\mathtt{M1}$; it specifies a matrix defining how variable(s) in the base model $\mathcal{M}$/$\mathtt{M0}$ are mapped onto a variable in the abstracted model $\mathcal{M'}$/$\mathtt{M1}$.

A tweaked implementation of the *Abstraction* class able to deal with these inputs has been written in *src/abstraction.py*. We import it.

In [4]:
from src.legacy.SCMMappings_1_0 import Abstraction

# Abstraction error refresher

Before moving to implementing a module for the automatic computation of error, let us review the definition of the abstraction error $e(\alpha)$.

Let's $\mathbf{A}$ and $\mathbf{B}$ two sets of variables in the base model $\mathcal{M}$/$\mathtt{M0}$; let's $X$ and $Y$ be the related variables in the abstracted model $\mathcal{M}'$/$\mathtt{M1}$. Let us now consider the following diagram:

$$
\begin{array}{ccc}
\mathcal{\mathcal{M}_{do}}\left[\mathbf{A}\right] & \overset{\mathcal{\mathcal{M}_{do}}\left[\phi_{\mathbf{B}}\right]}{\longrightarrow} & \mathcal{\mathcal{M}_{do}}\left[\mathbf{B}\right]\\
\sideset{}{\alpha_{X}}\downarrow &  & \sideset{}{\alpha_{Y}}\downarrow\\
\mathcal{\mathcal{M'}_{do}}\left[X\right] & \overset{\mathcal{\mathcal{M'}_{do}}\left[\phi_{Y}\right]}{\longrightarrow} & \mathcal{\mathcal{M'}_{do}}\left[Y\right]
\end{array}
$$

The abstraction error $E_\alpha(X,Y)$ with respect to the variables $X,Y$ is the JS distance under intervention between the upper path and the lower path:
$$
    D_{JS}(\alpha_Y \circ \mathcal{M}_{do}\left[\phi_{\mathbf{B}}\right], \mathcal{M'}_{do}\left[\phi_{Y}\right] \circ \alpha_X)
$$

The overall abstraction error $e(\alpha)$ is the highest abstraction error $E_\alpha(X,Y)$ considering all the meaningful pairs of variables $(X,Y)$ in the abstracted model $\mathcal{M}'$/$\mathtt{M1}$. We call the set of pair of variables $(X,Y)$ over which the error is estimated the **evaluation set** $\mathcal{J}$. 

## A note on the convention

To facilitate the understanding of the code and clarify how input models and abstractions should be defined, it is useful to write a short note on the representation convention for the matrices.

All morphisms we are dealing with (abstraction maps $\alpha_\cdot$ and mechanisms $\mathcal{M}[\phi_\cdot]$) are defined as stochastic matrices.

**Abstractions.** An abstraction $\alpha_X$ maps a set of variables $\mathbf{A}$ in the base model $\mathcal{M}$/$\mathtt{M0}$ to a single variable $X$ in the abstracted model $\mathcal{M}'$/$\mathtt{M1}$. 
Let $\mathcal{M}[A_1] \times \mathcal{M}[A_2] \times ... \times \mathcal{M}[A_n]$ be the domain of $\mathbf{A}$, and $\mathcal{M'}[X]$ the domain of $X$. Then, the matrix encoding $\alpha_X$ will be a two-dimensional matrix with dimensions:
$$
\left[\left|\mathcal{M'}[X]\right|, \left|\mathcal{M}[A_1]\right|\cdot\left|\mathcal{M}[A_2]\right|\cdot...\cdot\left|\mathcal{M}[A_n]\right|\right]
$$

For instance, consider $\alpha_{X}:A_1\times A_2\rightarrow X$ and suppose:
- $\mathcal{M}[A_1]=\{0,1,2\}$, 
- $\mathcal{M}[A_2]=\{0,1\}$, 
- $\mathcal{M'}[X]=\{0,1,2,3\}$

then, $\alpha_{X}$ is a stochastic matrix with dimensions $[4,6]$ and the following interpretation:
$$
\begin{array}{ccccccc}
 & \boldsymbol{(A_{1}=0,A_{2}=0)} & \boldsymbol{(A_{1}=0,A_{2}=1)} & \boldsymbol{(A_{1}=1,A_{2}=0)} & \boldsymbol{(A_{1}=1,A_{2}=1)} & \boldsymbol{(A_{1}=2,A_{2}=0)} & \boldsymbol{(A_{1}=2,A_{2}=1)}\\
\boldsymbol{(X=0)} & (0,0)\mapsto0 & (0,1)\mapsto0 & (1,0)\mapsto0 & (1,1)\mapsto0 & (2,0)\mapsto0 & (2,1)\mapsto0\\
\boldsymbol{(X=1)} & (0,0)\mapsto1 & (0,1)\mapsto1 & (1,0)\mapsto1 & (1,1)\mapsto1 & (2,0)\mapsto1 & (2,1)\mapsto1\\
\boldsymbol{(X=2)} & (0,0)\mapsto2 & (0,1)\mapsto2 & (1,0)\mapsto2 & (1,1)\mapsto2 & (2,0)\mapsto2 & (2,1)\mapsto2\\
\boldsymbol{(X=3)} & (0,0)\mapsto3 & (0,1)\mapsto3 & (1,0)\mapsto3 & (1,1)\mapsto3 & (2,0)\mapsto3 & (2,1)\mapsto3
\end{array}
$$

**Mechanisms.** A mechanism $\mathcal{M}[\phi_B]$ maps a set of variables $\mathbf{A}$ to a a set of variables $\mathbf{B}$ in the same model according to the conditional $P(\mathbf{B} \vert \mathbf{A})$. 
Let $\mathcal{M}[A_1] \times \mathcal{M}[A_2] \times ... \times \mathcal{M}[A_n]$ be the domain of $\mathbf{A}$, and $\mathcal{M}[B_1] \times \mathcal{M}[B_2] \times ... \times \mathcal{M}[B_m]$ the domain of $\mathbf{B}$. Then, the matrix encoding $\mathcal{M}[\phi_B]$ will be a two-dimensional matrix with dimensions:
$$
\left[\left|\mathcal{M}[B_1]\right|\cdot\left|\mathcal{M}[B_2]\right|\cdot...\cdot\left|\mathcal{M}[B_m]\right|, \left|\mathcal{M}[A_1]\right|\cdot\left|\mathcal{M}[A_2]\right|\cdot...\cdot\left|\mathcal{M}[A_n]\right|\right]
$$

For instance, consider $\mathcal{M}[\phi_B]:A_1\times A_2\rightarrow B_1\times B_2$ and suppose:
- $\mathcal{M}[A_1]=\{0,1,2\}$, 
- $\mathcal{M}[A_2]=\{0,1\}$, 
- $\mathcal{M}[B_1]=\{0,1\}$, 
- $\mathcal{M}[B_2]=\{0,1,2,3\}$, 

then, $\mathcal{M}[\phi_B]$ is a stochastic matrix with dimensions $[8,6]$ and the following interpretation:
$$
\begin{array}{ccccc}
 & \boldsymbol{(A_{1}=0,A_{2}=0)} & \boldsymbol{(A_{1}=0,A_{2}=1)} &  & \boldsymbol{(A_{1}=2,A_{2}=1)}\\
\boldsymbol{(B_{1}=0,B_{2}=0)} & P(B_{1}=0,B_{2}=0\vert A_{1}=0,A_{2}=0) & P(B_{1}=0,B_{2}=0\vert A_{1}=0,A_{2}=1) &  & P(B_{1}=0,B_{2}=0\vert A_{1}=2,A_{2}=1)\\
\boldsymbol{(B_{1}=0,B_{2}=1)} & P(B_{1}=0,B_{2}=1\vert A_{1}=0,A_{2}=0) & P(B_{1}=0,B_{2}=1\vert A_{1}=0,A_{2}=1) &  & P(B_{1}=0,B_{2}=1\vert A_{1}=2,A_{2}=1)\\
\boldsymbol{(B_{1}=0,B_{2}=2)} & P(B_{1}=0,B_{2}=2\vert A_{1}=0,A_{2}=0) & P(B_{1}=0,B_{2}=2\vert A_{1}=0,A_{2}=1) & ... & P(B_{1}=0,B_{2}=2\vert A_{1}=2,A_{2}=1)\\
\boldsymbol{(B_{1}=0,B_{2}=3)} & P(B_{1}=0,B_{2}=3\vert A_{1}=0,A_{2}=0) & P(B_{1}=0,B_{2}=3\vert A_{1}=0,A_{2}=1) &  & P(B_{1}=0,B_{2}=3\vert A_{1}=2,A_{2}=1)\\
\boldsymbol{(B_{1}=1,B_{2}=0)} & P(B_{1}=1,B_{2}=0\vert A_{1}=0,A_{2}=0) & P(B_{1}=1,B_{2}=0\vert A_{1}=0,A_{2}=1) &  & P(B_{1}=1,B_{2}=0\vert A_{1}=2,A_{2}=1)\\
\boldsymbol{(B_{1}=1,B_{2}=1)} & P(B_{1}=1,B_{2}=1\vert A_{1}=0,A_{2}=0) & P(B_{1}=1,B_{2}=1\vert A_{1}=0,A_{2}=1) &  & P(B_{1}=1,B_{2}=1\vert A_{1}=2,A_{2}=1)\\
\boldsymbol{(B_{1}=1,B_{2}=2)} & P(B_{1}=1,B_{2}=2\vert A_{1}=0,A_{2}=0) & P(B_{1}=1,B_{2}=2\vert A_{1}=0,A_{2}=1) &  & P(B_{1}=1,B_{2}=2\vert A_{1}=2,A_{2}=1)\\
\boldsymbol{(B_{1}=1,B_{2}=3)} & P(B_{1}=1,B_{2}=3\vert A_{1}=0,A_{2}=0) & P(B_{1}=1,B_{2}=3\vert A_{1}=0,A_{2}=1) &  & P(B_{1}=1,B_{2}=3\vert A_{1}=2,A_{2}=1)
\end{array}
$$

**Composition**. This encoding (codomain on the row, domain on the column; product of multiple finite sets in row-order) is consistent with the encoding used in *pgmpy* and it allows us to perform matrix multiplications in the same order of categorical composition.

# Evaluating abstraction considering only individual variables connected by a directed path

We start implementing our algorithm for the automatic computation of the abstraction error using as evaluation set $\mathcal{J}$ the pairs $(X,Y)$ such that there exist a directed path from $X$ to $Y$ in the abstracted model $\mathcal{M}'$/$\mathtt{M1}$.  

## Example 1

To drive our implementation, we work using as a reference the prototypical example we have studied in the previous notebooks. We consider models for the lung cancer scenario at different levels. A precise description of each one of them is available in the notebook *Categorical Abstraction.ipynb*.

### Model definition

We first define our two models. Notice how our models are now defined using *pgmpy*; we also take advantage of the *check_model()* function offered by *pgmpy* to verify that our model and its stochastic matrices are correct. 

In [5]:
M0 = BN([('Smoking','Tar'),('Tar','Cancer')])

cpdS = cpd(variable='Smoking',
          variable_card=2,
          values=[[.8],[.2]],
          evidence=None,
          evidence_card=None)
cpdT = cpd(variable='Tar',
          variable_card=2,
          values=[[1,.2],[0.,.8]],
          evidence=['Smoking'],
          evidence_card=[2])
cpdC = cpd(variable='Cancer',
          variable_card=2,
          values=[[.9,.6],[.1,.4]],
          evidence=['Tar'],
          evidence_card=[2])

M0.add_cpds(cpdS,cpdT,cpdC)
M0.check_model()

True

In [6]:
M1 = BN([('Smoking','Cancer')])

cpdS = cpd(variable='Smoking',
          variable_card=2,
          values=[[.8],[.2]],
          evidence=None,
          evidence_card=None)
cpdC = cpd(variable='Cancer',
          variable_card=2,
          values=[[.9,.66],[.1,.34]],
          evidence=['Smoking'],
          evidence_card=[2])

M1.add_cpds(cpdS,cpdC)
M1.check_model()

True

Next we define an abstraction. The definition of $(R,a,\alpha)$ has correspondingly changed to work with the *pgmpy* objects.

In [7]:
R = ['Smoking','Cancer']

a = {'Smoking': 'Smoking',
    'Cancer': 'Cancer'}
alphas = {'Smoking': np.eye(2),
         'Cancer': np.eye(2)}

In [8]:
A = Abstraction(M0,M1,R,a,alphas)

### Enumeration of all possible pairs in M1 with a directed path connection in M1

We now populate our evaluation set $\mathcal{J}$ computing all possible pairs (source,target) along directed paths in the abstracted model $\mathcal{M'}/\mathtt{M1}$. To do so, we call the *networkx* function *has_path()* among all possible pairs of nodes.

In [9]:
J = []

sources = list(A.M1.nodes())
targets = list(A.M1.nodes())

for s in sources:
    for t in list(set(targets)-{s}):
        if nx.has_path(A.M1,s,t):
            J.append((s,t))

In [10]:
print(J)

[('Smoking', 'Cancer')]


As obviosly clear, we have a single path in the abstracted model.

### Running the computational loop

We now come to the loop that will compute the abstraction error $e(\alpha)$. The loop iterates over all the pairs in the evaluation set $\mathcal{J}$ and computing the specific abstraction error.

Keeping in mind this diagram:
$$
\begin{array}{ccc}
\mathcal{\mathcal{M}_{do}}\left[\mathbf{A}\right] & \overset{\mathcal{\mathcal{M}_{do}}\left[\phi_{\mathbf{B}}\right]}{\longrightarrow} & \mathcal{\mathcal{M}_{do}}\left[\mathbf{B}\right]\\
\sideset{}{\alpha_{X}}\downarrow &  & \sideset{}{\alpha_{Y}}\downarrow\\
\mathcal{\mathcal{M'}_{do}}\left[X\right] & \overset{\mathcal{\mathcal{M'}_{do}}\left[\phi_{Y}\right]}{\longrightarrow} & \mathcal{\mathcal{M'}_{do}}\left[Y\right]
\end{array}
$$

let us decompose and explain the loop:
1. Given a pair from the evaluation set $\mathcal{J}$, extract the source node $X$ and the target node $Y$ in the abstracted model $\mathcal{M'}/\mathtt{M1}$.
2. Using the mapping $a$, retrieve the corresponding source set $\mathbf{A}$ and target set $\mathbf{B}$ in the base model $\mathcal{M}/\mathtt{M0}$.
3. Compute the intervention $do(X)$ in the abstracted model $\mathcal{M'}/\mathtt{M1}$ and initialize the *pgmpy* inference engine on this intervened model.
4. Compute the intervention $do(\mathbf{A})$ in the base model $\mathcal{M}/\mathtt{M0}$ and initialize the *pgmpy* inference engine on this intervened model.
5. Compute the mechanism $\mathcal{M'}_{do}[\phi_Y] = P_{{M'}_{do}}(Y\vert X)$ as $\frac{P_{{M'}_{do}}(Y,X)}{P_{{M'}_{do}}(X)}$. The two distributions $P_{{M'}_{do}}(Y,X)$ and $P_{{M'}_{do}}(X)$ are evaluated using the *pgmpy* inference engine.
4. Extract the matrix form of $\mathcal{M'}_{do}[\phi_Y] = P_{{M'}_{do}}(Y\vert X)$.
5. Check if the matrix correctly represents $Y$ on the rows and $X$ on the columns.
6. Compute the mechanism $\mathcal{M}_{do}[\phi_\mathbf{B}] = P_{{M}_{do}}(\mathbf{B}\vert \mathbf{A})$ as $\frac{P_{{M}_{do}}(\mathbf{B},\mathbf{A})}{P_{{M}_{do}}(\mathbf{A})}$. The two distributions $P_{{M}_{do}}(\mathbf{B},\mathbf{A})$ and $P_{{M}_{do}}(\mathbf{A})$ are evaluated using the *pgmpy* inference engine.
7. Extract the matrix form of $\mathcal{M}_{do}[\phi_\mathbf{B}] = P_{{M'}_{do}}(\mathbf{B}\vert \mathbf{A})$.
8. Reorder the matrix to be sure that the variables in $\mathbf{B}$ are coming before the variables in $\mathbf{A}$, and that they are in order they are supposed to be.
9. Compact the matrix reducing it to two dimensions: on the rows we have the variables of $\mathbf{B}$, on the columns the varibles of $\mathbf{A}$.
10. Extract the matrices for the abstractions of interest: $\alpha_X$ and $\alpha_Y$.
11. Compute the two alternative path on the diagram by composing $\mathcal{M'}_{do}[\phi_Y] \circ \alpha_X$ (lower path) and $\alpha_Y \circ \mathcal{M}_{do}[\phi_\mathbf{B}]$ (upper path) via a simple matrix product.
12. For every possible intervention on $\mathbf{A}$ compute the JS distance between the two paths. This is equivalent to consider every column in the matrix encoding of the two paths and computing their JS distance.
13. Select the highest distance with respect to all the interventions as the error $E_\alpha(Y,X)$.
14. Select the highest distance with repsect to all pairs $(X,Y)$ as the error $e(\alpha)$.

In [11]:
abstraction_errors = []

for pair in J:
    # Get nodes in the abstracted model
    M1_source = [pair[0]]
    M1_target = [pair[1]]
    print('\nM1: {0} -> {1}'.format(M1_source,M1_target))

    # Get nodes in the base model
    M0_source = A.invert_a(M1_source)
    M0_target = A.invert_a(M1_target)
    print('M0: {0} -> {1}'.format(M0_source,M0_target))
    
    # Perform intenrventions in the abstracted model and setup the inference engine
    M1do = A.M1.do(M1_source)
    inferM1 = VariableElimination(M1do)
    
    # Perform intenrventions in the base model and setup the inference engine
    M0do = A.M0.do(M0_source)
    inferM0 = VariableElimination(M0do)
    
    # Evaluate the mechanism in the abstracted model
    M1_joint_TS = inferM1.query(M1_target+M1_source,show_progress=False)
    M1_joint_S = inferM1.query(M1_source,show_progress=False)
    M1_cond_TS = M1_joint_TS / M1_joint_S
    
    # Extract the matrix
    M1_cond_TS_val = M1_cond_TS.values

    # Check ordering
    if (M1_cond_TS.variables[0] != M1_target[0]):
        M1_cond_TS_val = M1_cond_TS_val.T

    # Evaluate the mechanism in the base model
    M0_joint_TS = inferM0.query(M0_target+M0_source,show_progress=False)
    M0_joint_S = inferM0.query(M0_source,show_progress=False)
    M0_cond_TS = M0_joint_TS / M0_joint_S
    
    # Extract the matrix
    M0_cond_TS_val = M0_cond_TS.values

    # Reorder the matrix
    old_indexes = range(len(M0_target+M0_source))
    new_indexes = [(M0_target+M0_source).index(i) for i in M0_joint_TS.variables]
    M0_cond_TS_val = np.moveaxis(M0_cond_TS_val, old_indexes, new_indexes)

    # Compact the matrix
    M0_target_cards=[A.M0.get_cardinality(t) for t in M0_target]
    M0_target_card = np.prod(M0_target_cards)
    M0_source_cards=[A.M0.get_cardinality(s) for s in M0_source]
    M0_source_card = np.prod(M0_source_cards)
    M0_cond_TS_val = M0_cond_TS_val.reshape(M0_target_card,M0_source_card)
    
    # Extract the alphas
    alpha_S = A.alphas[M1_source[0]]
    alpha_T = A.alphas[M1_target[0]]
    
    # Evaluate the paths on the diagram
    lowerpath = np.dot(M1_cond_TS_val,alpha_S)
    upperpath = np.dot(alpha_T,M0_cond_TS_val)
    
    # Compute abstraction error for every possible intervention
    distances = []
    for c in range(lowerpath.shape[1]):
        distances.append( distance.jensenshannon(lowerpath[:,c],upperpath[:,c]) )
    print('All JS distances: {0}'.format(distances))
    
    # Select the greatest distance over all interventions
    print('\nAbstraction error: {0}'.format(np.max(distances)))
    abstraction_errors.append(np.max(distances))

# Select the greatest distance over all pairs considered
print('\n\nOVERALL ABSTRACTION ERROR: {0}'.format(np.max(abstraction_errors)))


M1: ['Smoking'] -> ['Cancer']
M0: ['Smoking'] -> ['Cancer']
All JS distances: [0.0, 4.344397704933063e-09]

Abstraction error: 4.344397704933063e-09


OVERALL ABSTRACTION ERROR: 4.344397704933063e-09


As we knew, this example is a case of zero-error abstraction, and indeed our result is virtually zero once we account for numerical approximation.

# Evaluating abstraction considering all pairs of nodes

We now move on using the same algorithm with a richer evaluation set $\mathcal{J}$ containg all the pairs $(X,Y)$ in the abstracted model $\mathcal{M}'$/$\mathtt{M1}$, independently from the existence of a directed path.

## Example 2

We still rely on the prototypical case study on the lung cancer scenario. This time we take into consideration the last extreme abstraction that removes all edges from the abstracted model. If we were to consider an evaluation set $\mathcal{J}$ that contains only directed paths in the abstracted model $\mathcal{M}''$/$\mathtt{M2}$ there would be no path; however in the notebook *Categorical Abstraction.ipynb* we computed a distance even in the absence of a directed path. For this reason in this example we enlarge the evaluation set $\mathcal{J}$.

### Model definition

First, we define the models and the abstraction.

In [12]:
M0 = BN([('Smoking','Cancer')])

cpdS = cpd(variable='Smoking',
          variable_card=2,
          values=[[.8],[.2]],
          evidence=None,
          evidence_card=None)
cpdC = cpd(variable='Cancer',
          variable_card=2,
          values=[[.9,.66],[.1,.34]],
          evidence=['Smoking'],
          evidence_card=[2])

M0.add_cpds(cpdS,cpdC)
M0.check_model()

True

In [13]:
M1 = BN()
M1.add_node('Smoking')
M1.add_node('Cancer')

cpdS = cpd(variable='Smoking',
          variable_card=2,
          values=[[.8],[.2]],
          evidence=None,
          evidence_card=None)
cpdC = cpd(variable='Cancer',
          variable_card=2,
          values=[[.852],[.148]],
          evidence=None,
          evidence_card=None)

M1.add_cpds(cpdS,cpdC)
M1.check_model()

True

In [14]:
R = ['Smoking','Cancer']

a = {'Smoking': 'Smoking',
    'Cancer': 'Cancer'}
alphas = {'Smoking': np.eye(2),
         'Cancer': np.eye(2)}

In [15]:
A = Abstraction(M0,M1,R,a,alphas)

### Enumeration of all possible pairs in M1

Then we populate the evaluation set $\mathcal{J}$ consider all possible pairs $(X,Y)$ in the abstracted model $\mathcal{M'}/\mathtt{M1}$. To do so, we just call *itertools.permutations()* to retrieve all the permutations of two elements.

In [16]:
J = list(itertools.permutations(A.M1.nodes(),2))

In [17]:
print(J)

[('Smoking', 'Cancer'), ('Cancer', 'Smoking')]


Notice that we consider the pair $(Smoking, Cancer)$ even if in the abstracted model there is no link between them. Moreover we consider also the pair $(Cancer, Smoking)$ although it is debatable whether we should consider this inverted relation that requires estimating a mechanism $\mathcal{M}[\phi_{Smoking}] = P(Smoking \vert Cancer)$ that hardly has a causal interpretation.

### Running the computational loop

We run the same evaluatin loop. See above for an explanation of every step.

In [18]:
abstraction_errors = []

for pair in J:
    # Get nodes in the abstracted model
    M1_source = [pair[0]]
    M1_target = [pair[1]]
    print('\nM1: {0} -> {1}'.format(M1_source,M1_target))

    # Get nodes in the base model
    M0_source = A.invert_a(M1_source)
    M0_target = A.invert_a(M1_target)
    print('M0: {0} -> {1}'.format(M0_source,M0_target))
    
    # Perform intenrventions in the abstracted model and setup the inference engine
    M1do = A.M1.do(M1_source)
    inferM1 = VariableElimination(M1do)
    
    # Perform intenrventions in the base model and setup the inference engine
    M0do = A.M0.do(M0_source)
    inferM0 = VariableElimination(M0do)
    
    # Evaluate the mechanism in the abstracted model
    M1_joint_TS = inferM1.query(M1_target+M1_source,show_progress=False)
    M1_joint_S = inferM1.query(M1_source,show_progress=False)
    M1_cond_TS = M1_joint_TS / M1_joint_S
    
    # Extract the matrix
    M1_cond_TS_val = M1_cond_TS.values

    # Check ordering
    if (M1_cond_TS.variables[0] != M1_target[0]):
        M1_cond_TS_val = M1_cond_TS_val.T

    # Evaluate the mechanism in the base model
    M0_joint_TS = inferM0.query(M0_target+M0_source,show_progress=False)
    M0_joint_S = inferM0.query(M0_source,show_progress=False)
    M0_cond_TS = M0_joint_TS / M0_joint_S
    
    # Extract the matrix
    M0_cond_TS_val = M0_cond_TS.values

    # Reorder the matrix
    old_indexes = range(len(M0_target+M0_source))
    new_indexes = [(M0_target+M0_source).index(i) for i in M0_joint_TS.variables]
    M0_cond_TS_val = np.moveaxis(M0_cond_TS_val, old_indexes, new_indexes)

    # Compact the matrix
    M0_target_cards=[A.M0.get_cardinality(t) for t in M0_target]
    M0_target_card = np.prod(M0_target_cards)
    M0_source_cards=[A.M0.get_cardinality(s) for s in M0_source]
    M0_source_card = np.prod(M0_source_cards)
    M0_cond_TS_val = M0_cond_TS_val.reshape(M0_target_card,M0_source_card)

    # Extract the alphas
    alpha_S = A.alphas[M1_source[0]]
    alpha_T = A.alphas[M1_target[0]]
    
    # Evaluate the paths on the diagram
    lowerpath = np.dot(M1_cond_TS_val,alpha_S)
    upperpath = np.dot(alpha_T,M0_cond_TS_val)
    
    # Compute abstraction error for every possible intervention
    distances = []
    for c in range(lowerpath.shape[1]):
        distances.append( distance.jensenshannon(lowerpath[:,c],upperpath[:,c]) )
    print('All JS distances: {0}'.format(distances))
    
    # Select the greatest distance over all interventions
    print('\nAbstraction error: {0}'.format(np.max(distances)))
    abstraction_errors.append(np.max(distances))

# Select the greatest distance over all pairs considered
print('\n\nOVERALL ABSTRACTION ERROR: {0}'.format(np.max(abstraction_errors)))


M1: ['Smoking'] -> ['Cancer']
M0: ['Smoking'] -> ['Cancer']
All JS distances: [0.051634404110767126, 0.15974085850231143]

Abstraction error: 0.15974085850231143

M1: ['Cancer'] -> ['Smoking']
M0: ['Cancer'] -> ['Smoking']
All JS distances: [0.0, 0.0]

Abstraction error: 0.0


OVERALL ABSTRACTION ERROR: 0.15974085850231143


Notice that, looking at the pair $(Smoking,Cancer)$ we obtain a result in line with what we obtained in the notebook *Categorical Abstraction.ipynb*. 
The result for the pair $(Cancer,Smoking)$ may look at first counterintuive: why the error is in this case equal to 0? The result actually make sense because we are measuring $P(Smoking\vert do(Cancer))$ in the two models $\mathcal{M}/M0$ and $\mathcal{M'}/M1$. In $\mathcal{M'}/M1$, $Smoking$ and $Cancer$ are independent from the beginning; in $\mathcal{M}/M0$, $Smoking$ and $Cancer$ are made independent by the intervention. So we are just comparing $P(Smoking)$ in $\mathcal{M}/M0$ and $\mathcal{M'}/M1$, and this marginal happens to have been defined equally in the two models.

Notice, also, that if we were to print $\mathcal{M'}[\phi_{Cancer}] = P(Cancer \vert Smoking)$ (or $\mathcal{M'}[\phi_{Smoking}] = P(Smoking \vert Cancer)$) we could see from its matrix form that it encodes a conditional distribution in which $Smoking$ and $Cancer$ are independent in the abstracted model $\mathcal{M'}/M1$. The matrix has the expected shape $[2,2]$ but it is columns are identical, suggesting that the value on the column does not matter. This makes sense ($Smoking$ and $Cancer$ have no connection in the graph, they are independent), and it is consistent with the formalism and result in the notebook *Categorical Abstraction.ipynb*.

# Evaluating abstraction considering pairs of node potentially having a causal relation

Considering an evaluation set $\mathcal{J}$ containing all the pairs $(X,Y)$ in the abstracted model $\mathcal{M}'$/$\mathtt{M1}$ seems to generate a set inclusing meaninglss pairs. We will then refine the evaluation set $\mathcal{J}$ containg all the pairs $(X,Y)$ that are connected by a directed path in $\mathcal{M}'$/$\mathtt{M1}$ or whose counterimages $(\mathbf{A},\mathbf{B})$ are connected by a directed path in $\mathcal{M}$/$\mathtt{M0}$.

## Example 3

We instantiate now a new test models containing more nodes and more challenging mappings.

### Model definition

First, we define the models and the abstraction.

In [19]:
def generate_values(c,d):
    val = np.random.rand(c,d)
    return val / np.sum(val,axis=0)

In [20]:
M0 = BN([('A','C'), ('E','B'), ('E','C'), ('B','C'), ('C','D'), ('C','F')])

cpdA = cpd(variable='A',
          variable_card=3,
          values=generate_values(3,1),
          evidence=None,
          evidence_card=None)

cpdE = cpd(variable='E',
          variable_card=2,
          values=generate_values(2,1),
          evidence=None,
          evidence_card=None)

cpdB = cpd(variable='B',
          variable_card=4,
          values=generate_values(4,2),
          evidence=['E'],
          evidence_card=[2])

cpdC = cpd(variable='C',
          variable_card=6,
          values=generate_values(6,24),
          evidence=['A','B','E'],
          evidence_card=[3,4,2])

cpdD = cpd(variable='D',
          variable_card=3,
          values=generate_values(3,6),
          evidence=['C'],
          evidence_card=[6])

cpdF = cpd(variable='F',
          variable_card=2,
          values=generate_values(2,6),
          evidence=['C'],
          evidence_card=[6])

M0.add_cpds(cpdA,cpdB,cpdC,cpdD,cpdE,cpdF)
M0.check_model()

True

In [21]:
M1 = BN([('X','Y'), ('Y','Z'), ('Y','W'), ('X','Z')])

cpdX = cpd(variable='X',
          variable_card=3,
          values=generate_values(3,1),
          evidence=None,
          evidence_card=None)

cpdY = cpd(variable='Y',
          variable_card=4,
          values=generate_values(4,3),
          evidence=['X'],
          evidence_card=[3])

cpdZ = cpd(variable='Z',
          variable_card=2,
          values=generate_values(2,12),
          evidence=['Y','X'],
          evidence_card=[4,3])

cpdW = cpd(variable='W',
          variable_card=2,
          values=generate_values(2,4),
          evidence=['Y'],
          evidence_card=[4])

M1.add_cpds(cpdX,cpdY,cpdZ,cpdW)
M1.check_model()

True

In [22]:
R = ['A','B', 'C', 'D', 'F']

a = {'A': 'X',
     'B': 'X',
     'C': 'Y',
     'D': 'Z',
     'F': 'W'}
alphas = {'X': generate_values(3,12),
         'Y': generate_values(4,6),
         'Z': generate_values(2,3),
         'W': generate_values(2,2)}

In [23]:
A = Abstraction(M0,M1,R,a,alphas)

### Enumeration of all possible pairs in M1 with a directed path connection in M1 or M0

We now populate our evaluation set $\mathcal{J}$ computing all possible pairs (source,target) along directed paths in the abstracted model $\mathcal{M'}/\mathtt{M1}$ or in the base model $\mathcal{M'}/\mathtt{M1}$. 

We start implementing a helper function *check_path_between_sets()* to check the existence of a path between two set of nodes $\mathbf{A}$ and $\mathbf{B}$. This function creates an artificial parent $A'$ to all the nodes in $\mathbf{A}$ and an artifical child $B'$ to all the nodes in $\mathbf{B}$ and then we call the *networkx* function *has_path()* between $A'$ and $B'$.

In [24]:
def check_path_between_sets(G,sources,targets):
    augmentedG = G.copy()
            
    augmented_s = 'augmented_s_'+str(np.random.randint(10**6))
    augmented_t = 'augmented_t_'+str(np.random.randint(10**6))
    augmentedG.add_node(augmented_s)
    augmentedG.add_node(augmented_t)

    [augmentedG.add_edge(augmented_s,s) for s in sources]
    [augmentedG.add_edge(t,augmented_t) for t in targets]

    return nx.has_path(augmentedG,augmented_s,augmented_t)  

We can populate $\mathcal{J}$. To verify if there is a path in $\mathcal{M'}/\mathtt{M1}$ between node $X$ and $Y$ we use the same method we used before of calling the *networkx* function *has_path()*. To verify if there is a path in $\mathcal{M}/\mathtt{M0}$ between the set of nodes $\mathbf{A}$ and $\mathbf{B}$ we rely on our helper function.

In [25]:
J = []

sources = list(A.M1.nodes())
targets = list(A.M1.nodes())

for s in sources:
    for t in list(set(targets)-{s}):
        if nx.has_path(A.M1,s,t):
            J.append((s,t))
        else:
            M0_sources = A.invert_a(s)
            M0_targets = A.invert_a(t)            
            if check_path_between_sets(A.M0,M0_sources,M0_targets):
                J.append((s,t))

In [26]:
print(J)

[('X', 'W'), ('X', 'Y'), ('X', 'Z'), ('Y', 'W'), ('Y', 'Z')]


### Running the computational loop

In [27]:
abstraction_errors = []

for pair in J:
    # Get nodes in the abstracted model
    M1_source = [pair[0]]
    M1_target = [pair[1]]
    print('\nM1: {0} -> {1}'.format(M1_source,M1_target))

    # Get nodes in the base model
    M0_source = A.invert_a(M1_source)
    M0_target = A.invert_a(M1_target)
    print('M0: {0} -> {1}'.format(M0_source,M0_target))
    
    # Perform intenrventions in the abstracted model and setup the inference engine
    M1do = A.M1.do(M1_source)
    inferM1 = VariableElimination(M1do)
    
    # Perform intenrventions in the base model and setup the inference engine
    M0do = A.M0.do(M0_source)
    inferM0 = VariableElimination(M0do)
    
    # Evaluate the mechanism in the abstracted model
    M1_joint_TS = inferM1.query(M1_target+M1_source,show_progress=False)
    M1_joint_S = inferM1.query(M1_source,show_progress=False)
    M1_cond_TS = M1_joint_TS / M1_joint_S
    
    # Extract the matrix
    M1_cond_TS_val = M1_cond_TS.values

    # Check ordering
    if (M1_cond_TS.variables[0] != M1_target[0]):
        M1_cond_TS_val = M1_cond_TS_val.T

    # Evaluate the mechanism in the base model
    M0_joint_TS = inferM0.query(M0_target+M0_source,show_progress=False)
    M0_joint_S = inferM0.query(M0_source,show_progress=False)
    M0_cond_TS = M0_joint_TS / M0_joint_S
    
    # Extract the matrix
    M0_cond_TS_val = M0_cond_TS.values

    # Reorder the matrix
    old_indexes = range(len(M0_target+M0_source))
    new_indexes = [(M0_target+M0_source).index(i) for i in M0_joint_TS.variables]
    M0_cond_TS_val = np.moveaxis(M0_cond_TS_val, old_indexes, new_indexes)

    # Compact the matrix
    M0_target_cards=[A.M0.get_cardinality(t) for t in M0_target]
    M0_target_card = np.prod(M0_target_cards)
    M0_source_cards=[A.M0.get_cardinality(s) for s in M0_source]
    M0_source_card = np.prod(M0_source_cards)
    M0_cond_TS_val = M0_cond_TS_val.reshape(M0_target_card,M0_source_card)

    # Extract the alphas
    alpha_S = A.alphas[M1_source[0]]
    alpha_T = A.alphas[M1_target[0]]
    
    # Evaluate the paths on the diagram
    lowerpath = np.dot(M1_cond_TS_val,alpha_S)
    upperpath = np.dot(alpha_T,M0_cond_TS_val)
    
    # Compute abstraction error for every possible intervention
    distances = []
    for c in range(lowerpath.shape[1]):
        distances.append( distance.jensenshannon(lowerpath[:,c],upperpath[:,c]) )
    print('All JS distances: {0}'.format(distances))
    
    # Select the greatest distance over all interventions
    print('\nAbstraction error: {0}'.format(np.max(distances)))
    abstraction_errors.append(np.max(distances))

# Select the greatest distance over all pairs considered
print('\n\nOVERALL ABSTRACTION ERROR: {0}'.format(np.max(abstraction_errors)))


M1: ['X'] -> ['W']
M0: ['A', 'B'] -> ['F']
All JS distances: [0.12521771836868564, 0.1098170956238661, 0.1093643467805497, 0.15438710717432574, 0.12046159176748646, 0.16917931812906775, 0.13779348059871385, 0.1727573437923306, 0.17731904977048296, 0.1549678911567308, 0.13311969480754737, 0.14345170601798582]

Abstraction error: 0.17731904977048296

M1: ['X'] -> ['Y']
M0: ['A', 'B'] -> ['C']
All JS distances: [0.11835877298107207, 0.19699602007864722, 0.09476904663959261, 0.20950728465444696, 0.15927935305699015, 0.14207969276805765, 0.11968508852041435, 0.1564581742712215, 0.17109467359446529, 0.14402290823110306, 0.1451752260206426, 0.15053887286820122]

Abstraction error: 0.20950728465444696

M1: ['X'] -> ['Z']
M0: ['A', 'B'] -> ['D']
All JS distances: [0.00906527140205899, 0.018891757270061746, 0.004833781584752415, 0.02302667236729111, 0.010767043207948444, 0.029256445382254467, 0.014593949677328466, 0.04728920146648553, 0.03818884145805786, 0.021735512048527937, 0.016418485883169

# Evaluating abstraction considering sets of node potentially having a causal relation

The actual definition of abstraction error requires considering not only all potential pairs $(X,Y)$ in the abstracted model $\mathcal{M}'$/$\mathtt{M1}$, but every disjoint set of variables $(\mathbf{X},\mathbf{Y})$ in $\mathcal{M}'$/$\mathtt{M1}$. Here, we will populate the evaluation set $\mathcal{J}$ containg with all the sets $(\mathbf{X},\mathbf{Y})$ that are connected by a directed path in $\mathcal{M}'$/$\mathtt{M1}$ or whose counterimages $(\mathbf{A},\mathbf{B})$ are connected by a directed path in $\mathcal{M}$/$\mathtt{M0}$.

## Example 4

We instantiate the same models as in Example 3.

### Model definition

We redefine the models and the abstraction.

In [28]:
M0 = BN([('A','C'), ('E','B'), ('E','C'), ('B','C'), ('C','D'), ('C','F')])

cpdA = cpd(variable='A',
          variable_card=3,
          values=generate_values(3,1),
          evidence=None,
          evidence_card=None)

cpdE = cpd(variable='E',
          variable_card=2,
          values=generate_values(2,1),
          evidence=None,
          evidence_card=None)

cpdB = cpd(variable='B',
          variable_card=4,
          values=generate_values(4,2),
          evidence=['E'],
          evidence_card=[2])

cpdC = cpd(variable='C',
          variable_card=6,
          values=generate_values(6,24),
          evidence=['A','B','E'],
          evidence_card=[3,4,2])

cpdD = cpd(variable='D',
          variable_card=3,
          values=generate_values(3,6),
          evidence=['C'],
          evidence_card=[6])

cpdF = cpd(variable='F',
          variable_card=2,
          values=generate_values(2,6),
          evidence=['C'],
          evidence_card=[6])

M0.add_cpds(cpdA,cpdB,cpdC,cpdD,cpdE,cpdF)
M0.check_model()

True

In [29]:
M1 = BN([('X','Y'), ('Y','Z'), ('Y','W'), ('X','Z')])

cpdX = cpd(variable='X',
          variable_card=3,
          values=generate_values(3,1),
          evidence=None,
          evidence_card=None)

cpdY = cpd(variable='Y',
          variable_card=4,
          values=generate_values(4,3),
          evidence=['X'],
          evidence_card=[3])

cpdZ = cpd(variable='Z',
          variable_card=2,
          values=generate_values(2,12),
          evidence=['Y','X'],
          evidence_card=[4,3])

cpdW = cpd(variable='W',
          variable_card=2,
          values=generate_values(2,4),
          evidence=['Y'],
          evidence_card=[4])

M1.add_cpds(cpdX,cpdY,cpdZ,cpdW)
M1.check_model()

True

In [30]:
R = ['A','B', 'C', 'D', 'F']

a = {'A': 'X',
     'B': 'X',
     'C': 'Y',
     'D': 'Z',
     'F': 'W'}
alphas = {'X': generate_values(3,12),
         'Y': generate_values(4,6),
         'Z': generate_values(2,3),
         'W': generate_values(2,2)}

In [31]:
A = Abstraction(M0,M1,R,a,alphas)

### Enumeration of all possible sets in M1 with directed path connections in M1 or M0

We now want to populate our evaluation set $\mathcal{J}$ considering not only pairs of nodes in M1, but all possible ways of generating two disjoint subsets out of $\mathcal{X}_\mathcal{M'}$.

To do so, we start implementing a simple function *powerset()* to compute the powersets of $\mathcal{X}_\mathcal{M'}$ following the recipe provided in the *itertools* documentation.

In [32]:
def powerset(iterable):
    s = list(iterable)
    return itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(len(s)+1))

We then populate the evaluation set $\mathcal{J}$ according to the following step:
1. Consider a pair of sets from the powerset
2. Check whether the sets are disjoint:
    - If yes, move to step 3
    - If not, go back to step 1
3. Check if the two sets of nodes have a directed path in \mathtt{M1}:
    - If yes, add them to $\mathcal{J}$ and go back to step 1
    - If not, go to step 4
4. Find the corresponding sets in M0 (Notice that since $a$ is a function we already know these two sets will also be disjoint)
5. Check if the two sets of nodes have a directed path in \mathtt{M0}:
    - If yes, add them to $\mathcal{J}$ and go back to step 1
    - If not, go back to step 1
    
This procedure evaluate all possible pairings of elements in the powerset, and it selects only the right ones for the evaluation set $\mathcal{J}$. 

In [33]:
J = []

sets = list(powerset(A.M1.nodes()))
sets.remove(())

for i in sets:
    for j in sets:
        M1_sources = list(i)
        M1_targets = list(j)
        if not(any(x in M1_sources for x in M1_targets)):            
            if check_path_between_sets(A.M1,M1_sources,M1_targets):
                print('- Checking {0} -> {1}: True'.format(M1_sources,M1_targets))
                J.append([M1_sources,M1_targets])
            else:
                print('- Checking {0} -> {1}: False'.format(M1_sources,M1_targets))
                M0_sources = A.invert_a(M1_sources)
                M0_targets = A.invert_a(M1_targets)
                if check_path_between_sets(A.M0,M0_sources,M0_targets):
                    print('---- Checking {0} -> {1}: True'.format(M0_sources,M0_targets))
                    J.append([M1_sources,M1_targets])
                else:
                    print('---- Checking {0} -> {1}: False'.format(M0_sources,M0_targets))
                        
print('\n {0} legitimate pairs of sets out of {1} possbile pairs of sets'.format(len(J),len(sets)**2))    

- Checking ['X'] -> ['Y']: True
- Checking ['X'] -> ['Z']: True
- Checking ['X'] -> ['W']: True
- Checking ['X'] -> ['Y', 'Z']: True
- Checking ['X'] -> ['Y', 'W']: True
- Checking ['X'] -> ['Z', 'W']: True
- Checking ['X'] -> ['Y', 'Z', 'W']: True
- Checking ['Y'] -> ['X']: False
---- Checking ['C'] -> ['A', 'B']: False
- Checking ['Y'] -> ['Z']: True
- Checking ['Y'] -> ['W']: True
- Checking ['Y'] -> ['X', 'Z']: True
- Checking ['Y'] -> ['X', 'W']: True
- Checking ['Y'] -> ['Z', 'W']: True
- Checking ['Y'] -> ['X', 'Z', 'W']: True
- Checking ['Z'] -> ['X']: False
---- Checking ['D'] -> ['A', 'B']: False
- Checking ['Z'] -> ['Y']: False
---- Checking ['D'] -> ['C']: False
- Checking ['Z'] -> ['W']: False
---- Checking ['D'] -> ['F']: False
- Checking ['Z'] -> ['X', 'Y']: False
---- Checking ['D'] -> ['A', 'B', 'C']: False
- Checking ['Z'] -> ['X', 'W']: False
---- Checking ['D'] -> ['A', 'B', 'F']: False
- Checking ['Z'] -> ['Y', 'W']: False
---- Checking ['D'] -> ['C', 'F']: False
-

In [34]:
print(J)

[[['X'], ['Y']], [['X'], ['Z']], [['X'], ['W']], [['X'], ['Y', 'Z']], [['X'], ['Y', 'W']], [['X'], ['Z', 'W']], [['X'], ['Y', 'Z', 'W']], [['Y'], ['Z']], [['Y'], ['W']], [['Y'], ['X', 'Z']], [['Y'], ['X', 'W']], [['Y'], ['Z', 'W']], [['Y'], ['X', 'Z', 'W']], [['X', 'Y'], ['Z']], [['X', 'Y'], ['W']], [['X', 'Y'], ['Z', 'W']], [['X', 'Z'], ['Y']], [['X', 'Z'], ['W']], [['X', 'Z'], ['Y', 'W']], [['X', 'W'], ['Y']], [['X', 'W'], ['Z']], [['X', 'W'], ['Y', 'Z']], [['Y', 'Z'], ['W']], [['Y', 'Z'], ['X', 'W']], [['Y', 'W'], ['Z']], [['Y', 'W'], ['X', 'Z']], [['X', 'Y', 'Z'], ['W']], [['X', 'Y', 'W'], ['Z']], [['X', 'Z', 'W'], ['Y']]]


### Running the computational loop

Running the computational loop is conceptually equivalent to what we have done so far, but we have now to deal with the fact that also when working within the high-level model we may have to consider sets of variables and not just individual nodes.

Instead of the diagram we have seen before:
$$
\begin{array}{ccc}
\mathcal{\mathcal{M}_{do}}\left[\mathbf{A}\right] & \overset{\mathcal{\mathcal{M}_{do}}\left[\phi_{\mathbf{B}}\right]}{\longrightarrow} & \mathcal{\mathcal{M}_{do}}\left[\mathbf{B}\right]\\
\sideset{}{\alpha_{X}}\downarrow &  & \sideset{}{\alpha_{Y}}\downarrow\\
\mathcal{\mathcal{M'}_{do}}\left[X\right] & \overset{\mathcal{\mathcal{M'}_{do}}\left[\phi_{Y}\right]}{\longrightarrow} & \mathcal{\mathcal{M'}_{do}}\left[Y\right]
\end{array}
$$
we have to consider:
$$
\begin{array}{ccc}
\mathcal{\mathcal{M}_{do}}\left[\mathbf{A}\right] & \overset{\mathcal{\mathcal{M}_{do}}\left[\phi_{\mathbf{B}}\right]}{\longrightarrow} & \mathcal{\mathcal{M}_{do}}\left[\mathbf{B}\right]\\
\sideset{}{\alpha_{\mathbf{X}}}\downarrow &  & \sideset{}{\alpha_{\mathbf{Y}}}\downarrow\\
\mathcal{\mathcal{M'}_{do}}\left[\mathbf{X}\right] & \overset{\mathcal{\mathcal{M'}_{do}}\left[\phi_{\mathbf{Y}}\right]}{\longrightarrow} & \mathcal{\mathcal{M'}_{do}}\left[\mathbf{Y}\right]
\end{array}
$$
where $X$ and $Y$ has now been typed in boldface, $\mathbf{X},\mathbf{Y}$ to denote we are working with sets.

This requires us assemble abstractions ($\alpha_{\mathbf{X}},\alpha_{\mathbf{Y}}$) and mechanisms ($\mathcal{M}[\phi_{\mathbf{Y}}]$) with the proper dimensionality. To do so we introduce two helper functions:
- *tensorize_list*: given a list of matrices $[X_1, X_2, X_3, ..., X_n]$ with dimensions $[(r_1,c_1),(r_2,c_2),(r_3,c_3),...,(r_n,c_n)]$, this function recursively compute the tensor product $X_1 \otimes X_2 \otimes X_3 \otimes ... \otimes X_n$ with dimension $(r_1\cdot r_2\cdot r_3\cdot ...\cdot r_n, c_1\cdot c_2\cdot c_3\cdot ...\cdot c_n)$.
- *tensorize_mechanism*: this function wraps up the code we used previously to compute the mechanism in $\mathtt{M0}$. It receives the inference engine for the underlying graph, a source set of nodes, a target set of nodes, and a dictionary with the cardinalities of the nodes. It then use the engine to evaluate the mechanism, and the it re-orders and compacts the resulting matrix in the right shape.

In [35]:
def tensorize_list(tensor,l):
    if tensor is None:
        if len(l)>1:
            tensor = np.einsum('ij,kl->ikjl',l[0],l[1])
            tensor = tensor.reshape((tensor.shape[0]*tensor.shape[1],tensor.shape[2]*tensor.shape[3]))
            return tensorize_list(tensor,l[2:])
        else:
            return l[0]
    else:
        if len(l)>0:
            tensor = np.einsum('ij,kl->ikjl',tensor,l[0])
            tensor = tensor.reshape((tensor.shape[0]*tensor.shape[1],tensor.shape[2]*tensor.shape[3]))
            return tensorize_list(tensor,l[1:])
        else:
            return tensor

In [36]:
def tensorize_mechanisms(inference,sources,targets,cardinalities):
    
    # Evaluate the mechanism
    joint_TS = inference.query(targets+sources,show_progress=False)
    marginal_S = inference.query(sources,show_progress=False)
    cond_TS = joint_TS / marginal_S
    
    # Extract the matrix
    cond_TS_val = cond_TS.values

    # Reorder the matrix
    old_indexes = range(len(targets+sources))
    new_indexes = [(targets+sources).index(i) for i in joint_TS.variables]
    cond_TS_val = np.moveaxis(cond_TS_val, old_indexes, new_indexes)

    # Compact the matrix
    target_cards=[cardinalities[t] for t in targets]
    target_card = np.prod(target_cards)
    source_cards=[cardinalities[s] for s in sources]
    source_card = np.prod(source_cards)
    cond_TS_val = cond_TS_val.reshape(target_card,source_card)
    
    return cond_TS_val

We can then run the actual computational loop. Its overall organization is the same, although we rely on the functions defined above to make it more compact:

1. Given a pair from the evaluation set $\mathcal{J}$, extract the source nodes $\mathbf{X}$ and the target nodes $\mathbf{Y}$ in the abstracted model $\mathcal{M'}/\mathtt{M1}$.
2. Using the mapping $a$, retrieve the corresponding source set $\mathbf{A}$ and target set $\mathbf{B}$ in the base model $\mathcal{M}/\mathtt{M0}$.
3. Compute the intervention $do(X)$ in the abstracted model $\mathcal{M'}/\mathtt{M1}$ and initialize the *pgmpy* inference engine on this intervened model.
4. Compute the intervention $do(\mathbf{A})$ in the base model $\mathcal{M}/\mathtt{M0}$ and initialize the *pgmpy* inference engine on this intervened model.
5. Compute the mechanism $\mathcal{M'}_{do}[\phi_\mathbf{Y}]$ relying on *tensorize_mechanism()*. 
6. Compute the mechanism $\mathcal{M}_{do}[\phi_\mathbf{B}]$ relying on *tensorize_mechanism()*. 
7. Compute the matrix for $\alpha_\mathbf{X}$ relying on *tensorize_list()*.
8. Compute the matrix for $\alpha_\mathbf{Y}$ relying on *tensorize_list()*.
9. Compute the two alternative path on the diagram by composing $\mathcal{M'}_{do}[\phi_\mathbf{Y}] \circ \alpha_\mathbf{X}$ (lower path) and $\alpha_\mathbf{Y} \circ \mathcal{M}_{do}[\phi_\mathbf{B}]$ (upper path) via a simple matrix product.
10. For every possible intervention on $\mathbf{A}$ compute the JS distance between the two paths.
11. Select the highest distance with respect to all the interventions as the error $E_\alpha(\mathbf{Y},\mathbf{X})$.
12. Select the highest distance with repsect to all pairs $(\mathbf{X},\mathbf{Y})$ as the error $e(\alpha)$.

In [37]:
abstraction_errors = []

for pair in J:
    # Get nodes in the abstracted model
    M1_sources = pair[0]
    M1_targets = pair[1]
    print('\nM1: {0} -> {1}'.format(M1_sources,M1_targets))

    # Get nodes in the base model
    M0_sources = A.invert_a(M1_sources)
    M0_targets = A.invert_a(M1_targets)
    print('M0: {0} -> {1}'.format(M0_sources,M0_targets))
    
    # Perform interventions in the abstracted model and setup the inference engine
    M1do = A.M1.do(M1_sources)
    inferM1 = VariableElimination(M1do)
    
    # Perform interventions in the base model and setup the inference engine
    M0do = A.M0.do(M0_sources)
    inferM0 = VariableElimination(M0do)

    # Compute the high-level mechanisms
    M1_cond_TS_val = tensorize_mechanisms(inferM1,M1_sources,M1_targets,A.M1.get_cardinality())
    print('M1 mechanism shape: {}'.format(M1_cond_TS_val.shape))
        
    # Compute the low-level mechanisms
    M0_cond_TS_val = tensorize_mechanisms(inferM0,M0_sources,M0_targets,A.M0.get_cardinality())
    print('M0 mechanism shape: {}'.format(M0_cond_TS_val.shape))

    # Compute the alpha for sources
    alphas_S = [A.alphas[i] for i in M1_sources]
    alpha_S = tensorize_list(None,alphas_S)
    print('Alpha_s shape: {}'.format(alpha_S.shape))
    
    # Compute the alpha for targers
    alphas_T = [A.alphas[i] for i in M1_targets]
    alpha_T = tensorize_list(None,alphas_T)
    print('Alpha_t shape: {}'.format(alpha_T.shape))
        
    # Evaluate the paths on the diagram
    lowerpath = np.dot(M1_cond_TS_val,alpha_S)
    upperpath = np.dot(alpha_T,M0_cond_TS_val)
    
    # Compute abstraction error for every possible intervention
    distances = []
    for c in range(lowerpath.shape[1]):
        distances.append( distance.jensenshannon(lowerpath[:,c],upperpath[:,c]) )
    print('All JS distances: {0}'.format(distances))
    
    # Select the greatest distance over all interventions
    print('\nAbstraction error: {0}'.format(np.max(distances)))
    abstraction_errors.append(np.max(distances))

# Select the greatest distance over all pairs considered
print('\n\nOVERALL ABSTRACTION ERROR: {0}'.format(np.max(abstraction_errors)))


M1: ['X'] -> ['Y']
M0: ['A', 'B'] -> ['C']
M1 mechanism shape: (4, 3)
M0 mechanism shape: (6, 12)
Alpha_s shape: (3, 12)
Alpha_t shape: (4, 6)
All JS distances: [0.14852606329307985, 0.09265621144301286, 0.12001084898732219, 0.15997989391107853, 0.16203192810476508, 0.12696680980769243, 0.10069809180846691, 0.1863697329033639, 0.1047695132817974, 0.11807985631483506, 0.1179253134973884, 0.18437104992516792]

Abstraction error: 0.1863697329033639

M1: ['X'] -> ['Z']
M0: ['A', 'B'] -> ['D']
M1 mechanism shape: (2, 3)
M0 mechanism shape: (3, 12)
Alpha_s shape: (3, 12)
Alpha_t shape: (2, 3)
All JS distances: [0.0351304898487227, 0.06733630947989032, 0.0313707716587073, 0.0329667651770552, 0.03126112349484746, 0.03158249424275995, 0.047243008500498236, 0.0019017729573616153, 0.03424179787827428, 0.02209397224515851, 0.023474779635804806, 0.04781008084384352]

Abstraction error: 0.06733630947989032

M1: ['X'] -> ['W']
M0: ['A', 'B'] -> ['F']
M1 mechanism shape: (2, 3)
M0 mechanism shape: (2

M0 mechanism shape: (2, 36)
Alpha_s shape: (6, 36)
Alpha_t shape: (2, 2)
All JS distances: [0.03067203491489729, 0.030672034914896472, 0.030672034914896895, 0.045551698121225746, 0.04555169812122603, 0.045551698121225746, 0.04748251494799353, 0.047482514947992945, 0.047482514947992945, 0.0409845237818843, 0.040984523781884684, 0.04098452378188504, 0.041699048049717226, 0.041699048049717226, 0.041699048049717205, 0.04609574520099206, 0.04609574520099146, 0.04609574520099236, 0.04539146222920569, 0.04539146222920508, 0.045391462229205155, 0.04707914958024961, 0.04707914958024957, 0.04707914958024961, 0.04617497146167895, 0.04617497146167951, 0.04617497146167917, 0.04102381152514819, 0.04102381152514891, 0.041023811525147516, 0.04440594095298096, 0.044405940952980685, 0.044405940952980984, 0.04921460205222507, 0.04921460205222507, 0.049214602052225276]

Abstraction error: 0.049214602052225276

M1: ['X', 'Z'] -> ['Y', 'W']
M0: ['A', 'B', 'D'] -> ['C', 'F']
M1 mechanism shape: (8, 6)
M0 mec

# Automating the evaluation of abstraction error

To finalize the process of automation we package and move our code inside the class *Abstraction* in *src/pgm_models.py*. We define a function *evaluate_abstraction_error()* which, besides a verbosity parameter, takes two key parameters:

- *metric*: a function defining how to compute the distance among interventional distribution. If no function is passed, the algorithm falls back on JSD.
- *J_algorithm*: a function used to compute the evaluation set $\mathcal{J}$. If no function is passed, the algorithms falls back on the last algorithm we defined in this notebook (enumerating all pairs for which there is a directed path in the base or in the abstracted model).

These two parameters are key component for evaluating abstraction error, determining the quality and the computational cost of the process. In particular, the *J_algorithm* determines the cardinality of the set of pairs to evaluate and therefore has a direct impact on the complexity of the algorithm. Heuristics may be used to reduce this set and allow an efficient computation.

## Example 5

We now look at a last, even more complex abstract case, and use our *Abstraction* object.

### Model definition

We will use the exact same model we implemented above.

### Running the computational loop

In [38]:
result = A.evaluate_abstraction_error(verbose=True)

- Checking ['X'] -> ['Y']: True
- Checking ['X'] -> ['Z']: True
- Checking ['X'] -> ['W']: True
- Checking ['X'] -> ['Y', 'Z']: True
- Checking ['X'] -> ['Y', 'W']: True
- Checking ['X'] -> ['Z', 'W']: True
- Checking ['X'] -> ['Y', 'Z', 'W']: True
- Checking ['Y'] -> ['X']: False
---- Checking ['C'] -> ['A', 'B']: False
- Checking ['Y'] -> ['Z']: True
- Checking ['Y'] -> ['W']: True
- Checking ['Y'] -> ['X', 'Z']: True
- Checking ['Y'] -> ['X', 'W']: True
- Checking ['Y'] -> ['Z', 'W']: True
- Checking ['Y'] -> ['X', 'Z', 'W']: True
- Checking ['Z'] -> ['X']: False
---- Checking ['D'] -> ['A', 'B']: False
- Checking ['Z'] -> ['Y']: False
---- Checking ['D'] -> ['C']: False
- Checking ['Z'] -> ['W']: False
---- Checking ['D'] -> ['F']: False
- Checking ['Z'] -> ['X', 'Y']: False
---- Checking ['D'] -> ['A', 'B', 'C']: False
- Checking ['Z'] -> ['X', 'W']: False
---- Checking ['D'] -> ['A', 'B', 'F']: False
- Checking ['Z'] -> ['Y', 'W']: False
---- Checking ['D'] -> ['C', 'F']: False
-

M0 mechanism shape: (6, 36)
Alpha_s shape: (6, 36)
Alpha_t shape: (4, 6)
All JS distances: [0.14852606329307957, 0.1485260632930798, 0.14852606329307955, 0.09265621144301266, 0.09265621144301273, 0.0926562114430127, 0.12001084898732221, 0.12001084898732214, 0.12001084898732231, 0.1599798939110785, 0.15997989391107822, 0.15997989391107834, 0.1620319281047652, 0.16203192810476508, 0.16203192810476508, 0.12696680980769237, 0.1269668098076927, 0.12696680980769257, 0.10069809180846705, 0.10069809180846673, 0.10069809180846714, 0.18636973290336362, 0.18636973290336362, 0.1863697329033639, 0.1047695132817973, 0.10476951328179758, 0.10476951328179743, 0.11807985631483495, 0.11807985631483514, 0.11807985631483495, 0.11792531349738826, 0.11792531349738844, 0.11792531349738834, 0.18437104992516795, 0.18437104992516803, 0.1843710499251681]

Abstraction error: 0.1863697329033639

M1: ['X', 'Z'] -> ['W']
M0: ['A', 'B', 'D'] -> ['F']
M1 mechanism shape: (2, 6)
M0 mechanism shape: (2, 36)
Alpha_s shap

M1 mechanism shape: (4, 12)
M0 mechanism shape: (6, 72)
Alpha_s shape: (12, 72)
Alpha_t shape: (4, 6)
All JS distances: [0.14852606329307946, 0.1485260632930798, 0.14852606329307974, 0.14852606329307969, 0.1485260632930798, 0.14852606329307946, 0.09265621144301282, 0.09265621144301286, 0.09265621144301267, 0.09265621144301288, 0.09265621144301284, 0.09265621144301305, 0.12001084898732209, 0.12001084898732223, 0.12001084898732194, 0.12001084898732232, 0.12001084898732234, 0.12001084898732232, 0.15997989391107822, 0.15997989391107814, 0.15997989391107836, 0.15997989391107853, 0.15997989391107834, 0.1599798939110782, 0.16203192810476505, 0.1620319281047651, 0.1620319281047651, 0.1620319281047652, 0.16203192810476527, 0.1620319281047651, 0.1269668098076927, 0.12696680980769232, 0.12696680980769245, 0.1269668098076926, 0.12696680980769265, 0.1269668098076925, 0.10069809180846702, 0.10069809180846702, 0.10069809180846678, 0.10069809180846685, 0.10069809180846676, 0.10069809180846669, 0.18636

## Example 6

We instantiate a last complex abstraction.

### Model definition

We redefine the models and the abstraction.

In [39]:
M0 = BN([('A','C'), ('A','D'), ('B','D'), ('B','E'), ('B','F'), ('F','I'), ('D','H'), ('G','E'), ('G','F')])

cpdA = cpd(variable='A',
          variable_card=3,
          values=generate_values(3,1),
          evidence=None,
          evidence_card=None)

cpdB = cpd(variable='B',
          variable_card=4,
          values=generate_values(4,1),
          evidence=None,
          evidence_card=None)

cpdC = cpd(variable='C',
          variable_card=7,
          values=generate_values(7,3),
          evidence=['A'],
          evidence_card=[3])

cpdD = cpd(variable='D',
          variable_card=4,
          values=generate_values(4,12),
          evidence=['A','B'],
          evidence_card=[3,4])

cpdE = cpd(variable='E',
          variable_card=3,
          values=generate_values(3,8),
          evidence=['B','G'],
          evidence_card=[4,2])

cpdF = cpd(variable='F',
          variable_card=3,
          values=generate_values(3,8),
          evidence=['B','G'],
          evidence_card=[4,2])

cpdG = cpd(variable='G',
          variable_card=2,
          values=generate_values(2,1),
          evidence=None,
          evidence_card=None)

cpdH = cpd(variable='H',
          variable_card=6,
          values=generate_values(6,4),
          evidence=['D'],
          evidence_card=[4])

cpdI = cpd(variable='I',
          variable_card=5,
          values=generate_values(5,3),
          evidence=['F'],
          evidence_card=[3])

M0.add_cpds(cpdA,cpdB,cpdC,cpdD,cpdE,cpdF,cpdG,cpdH,cpdI)
M0.check_model()

True

In [40]:
M1 = BN([('X','Z'), ('Y','W'), ('Y','V')])
M1.add_node('U')

cpdX = cpd(variable='X',
          variable_card=3,
          values=generate_values(3,1),
          evidence=None,
          evidence_card=None)

cpdY = cpd(variable='Y',
          variable_card=3,
          values=generate_values(3,1),
          evidence=None,
          evidence_card=None)

cpdZ = cpd(variable='Z',
          variable_card=24,
          values=generate_values(24,3),
          evidence=['X'],
          evidence_card=[3])

cpdW = cpd(variable='W',
          variable_card=2,
          values=generate_values(2,3),
          evidence=['Y'],
          evidence_card=[3])

cpdV = cpd(variable='V',
          variable_card=8,
          values=generate_values(8,3),
          evidence=['Y'],
          evidence_card=[3])

cpdU = cpd(variable='U',
          variable_card=2,
          values=generate_values(2,1),
          evidence=None,
          evidence_card=None)

M1.add_cpds(cpdX,cpdY,cpdZ,cpdW,cpdV,cpdU)
M1.check_model()

True

In [41]:
R = ['A','B', 'D', 'E', 'F', 'G', 'H', 'I']

a = {'A': 'X',
     'B': 'Y',
     'D': 'Z',
     'E': 'W',
     'F': 'V',
     'G': 'U',
     'H': 'Z',
     'I': 'V'}
alphas = {'X': generate_values(3,3),
         'Y': generate_values(3,4),
         'Z': generate_values(24,24),
         'W': generate_values(2,3),
         'V': generate_values(8,15),
         'U': generate_values(2,2),
         }

In [42]:
A = Abstraction(M0,M1,R,a,alphas)

### Running the computational loop

In [43]:
result = A.evaluate_abstraction_error(verbose=True)

- Checking ['X'] -> ['Z']: True
- Checking ['X'] -> ['Y']: False
---- Checking ['A'] -> ['B']: False
- Checking ['X'] -> ['W']: False
---- Checking ['A'] -> ['E']: False
- Checking ['X'] -> ['V']: False
---- Checking ['A'] -> ['F', 'I']: False
- Checking ['X'] -> ['U']: False
---- Checking ['A'] -> ['G']: False
- Checking ['X'] -> ['Z', 'Y']: True
- Checking ['X'] -> ['Z', 'W']: True
- Checking ['X'] -> ['Z', 'V']: True
- Checking ['X'] -> ['Z', 'U']: True
- Checking ['X'] -> ['Y', 'W']: False
---- Checking ['A'] -> ['B', 'E']: False
- Checking ['X'] -> ['Y', 'V']: False
---- Checking ['A'] -> ['B', 'F', 'I']: False
- Checking ['X'] -> ['Y', 'U']: False
---- Checking ['A'] -> ['B', 'G']: False
- Checking ['X'] -> ['W', 'V']: False
---- Checking ['A'] -> ['E', 'F', 'I']: False
- Checking ['X'] -> ['W', 'U']: False
---- Checking ['A'] -> ['E', 'G']: False
- Checking ['X'] -> ['V', 'U']: False
---- Checking ['A'] -> ['F', 'G', 'I']: False
- Checking ['X'] -> ['Z', 'Y', 'W']: True
- Checki

---- Checking ['F', 'I'] -> ['D', 'H']: False
- Checking ['V'] -> ['Y']: False
---- Checking ['F', 'I'] -> ['B']: False
- Checking ['V'] -> ['W']: False
---- Checking ['F', 'I'] -> ['E']: False
- Checking ['V'] -> ['U']: False
---- Checking ['F', 'I'] -> ['G']: False
- Checking ['V'] -> ['X', 'Z']: False
---- Checking ['F', 'I'] -> ['A', 'D', 'H']: False
- Checking ['V'] -> ['X', 'Y']: False
---- Checking ['F', 'I'] -> ['A', 'B']: False
- Checking ['V'] -> ['X', 'W']: False
---- Checking ['F', 'I'] -> ['A', 'E']: False
- Checking ['V'] -> ['X', 'U']: False
---- Checking ['F', 'I'] -> ['A', 'G']: False
- Checking ['V'] -> ['Z', 'Y']: False
---- Checking ['F', 'I'] -> ['B', 'D', 'H']: False
- Checking ['V'] -> ['Z', 'W']: False
---- Checking ['F', 'I'] -> ['D', 'E', 'H']: False
- Checking ['V'] -> ['Z', 'U']: False
---- Checking ['F', 'I'] -> ['D', 'G', 'H']: False
- Checking ['V'] -> ['Y', 'W']: False
---- Checking ['F', 'I'] -> ['B', 'E']: False
- Checking ['V'] -> ['Y', 'U']: False
--

- Checking ['X', 'U'] -> ['V']: False
---- Checking ['A', 'G'] -> ['F', 'I']: True
- Checking ['X', 'U'] -> ['Z', 'Y']: True
- Checking ['X', 'U'] -> ['Z', 'W']: True
- Checking ['X', 'U'] -> ['Z', 'V']: True
- Checking ['X', 'U'] -> ['Y', 'W']: False
---- Checking ['A', 'G'] -> ['B', 'E']: True
- Checking ['X', 'U'] -> ['Y', 'V']: False
---- Checking ['A', 'G'] -> ['B', 'F', 'I']: True
- Checking ['X', 'U'] -> ['W', 'V']: False
---- Checking ['A', 'G'] -> ['E', 'F', 'I']: True
- Checking ['X', 'U'] -> ['Z', 'Y', 'W']: True
- Checking ['X', 'U'] -> ['Z', 'Y', 'V']: True
- Checking ['X', 'U'] -> ['Z', 'W', 'V']: True
- Checking ['X', 'U'] -> ['Y', 'W', 'V']: False
---- Checking ['A', 'G'] -> ['B', 'E', 'F', 'I']: True
- Checking ['X', 'U'] -> ['Z', 'Y', 'W', 'V']: True
- Checking ['Z', 'Y'] -> ['X']: False
---- Checking ['B', 'D', 'H'] -> ['A']: False
- Checking ['Z', 'Y'] -> ['W']: True
- Checking ['Z', 'Y'] -> ['V']: True
- Checking ['Z', 'Y'] -> ['U']: False
---- Checking ['B', 'D', 

---- Checking ['E', 'G'] -> ['D', 'H']: False
- Checking ['W', 'U'] -> ['Y']: False
---- Checking ['E', 'G'] -> ['B']: False
- Checking ['W', 'U'] -> ['V']: False
---- Checking ['E', 'G'] -> ['F', 'I']: True
- Checking ['W', 'U'] -> ['X', 'Z']: False
---- Checking ['E', 'G'] -> ['A', 'D', 'H']: False
- Checking ['W', 'U'] -> ['X', 'Y']: False
---- Checking ['E', 'G'] -> ['A', 'B']: False
- Checking ['W', 'U'] -> ['X', 'V']: False
---- Checking ['E', 'G'] -> ['A', 'F', 'I']: True
- Checking ['W', 'U'] -> ['Z', 'Y']: False
---- Checking ['E', 'G'] -> ['B', 'D', 'H']: False
- Checking ['W', 'U'] -> ['Z', 'V']: False
---- Checking ['E', 'G'] -> ['D', 'F', 'H', 'I']: True
- Checking ['W', 'U'] -> ['Y', 'V']: False
---- Checking ['E', 'G'] -> ['B', 'F', 'I']: True
- Checking ['W', 'U'] -> ['X', 'Z', 'Y']: False
---- Checking ['E', 'G'] -> ['A', 'B', 'D', 'H']: False
- Checking ['W', 'U'] -> ['X', 'Z', 'V']: False
---- Checking ['E', 'G'] -> ['A', 'D', 'F', 'H', 'I']: True
- Checking ['W', 'U

---- Checking ['D', 'E', 'G', 'H'] -> ['A', 'B', 'F', 'I']: True
- Checking ['Z', 'V', 'U'] -> ['X']: False
---- Checking ['D', 'F', 'G', 'H', 'I'] -> ['A']: False
- Checking ['Z', 'V', 'U'] -> ['Y']: False
---- Checking ['D', 'F', 'G', 'H', 'I'] -> ['B']: False
- Checking ['Z', 'V', 'U'] -> ['W']: False
---- Checking ['D', 'F', 'G', 'H', 'I'] -> ['E']: True
- Checking ['Z', 'V', 'U'] -> ['X', 'Y']: False
---- Checking ['D', 'F', 'G', 'H', 'I'] -> ['A', 'B']: False
- Checking ['Z', 'V', 'U'] -> ['X', 'W']: False
---- Checking ['D', 'F', 'G', 'H', 'I'] -> ['A', 'E']: True
- Checking ['Z', 'V', 'U'] -> ['Y', 'W']: False
---- Checking ['D', 'F', 'G', 'H', 'I'] -> ['B', 'E']: True
- Checking ['Z', 'V', 'U'] -> ['X', 'Y', 'W']: False
---- Checking ['D', 'F', 'G', 'H', 'I'] -> ['A', 'B', 'E']: True
- Checking ['Y', 'W', 'V'] -> ['X']: False
---- Checking ['B', 'E', 'F', 'I'] -> ['A']: False
- Checking ['Y', 'W', 'V'] -> ['Z']: False
---- Checking ['B', 'E', 'F', 'I'] -> ['D', 'H']: True
- Ch

...

All JS distances: [0.14671651019565887, 0.1467165101956589, 0.1467165101956589, 0.14671651019565882, 0.14671651019565893, 0.1467165101956589, 0.14671651019565893, 0.1467165101956589, 0.14671651019565898, 0.14671651019565898, 0.1467165101956589, 0.14671651019565893, 0.14671651019565882, 0.14671651019565893, 0.14671651019565893, 0.1467165101956589, 0.14671651019565882, 0.14671651019565896, 0.14671651019565893, 0.14671651019565893, 0.1467165101956589, 0.14671651019565887, 0.14671651019565896, 0.14671651019565898, 0.14671651019565898, 0.14671651019565898, 0.14671651019565887, 0.14671651019565893, 0.146716510195659, 0.14671651019565898, 0.1467165101956589, 0.14671651019565893, 0.1467165101956589, 0.14671651019565893, 0.14671651019565898, 0.14671651019565893, 0.14671651019565898, 0.1467165101956589, 0.14671651019565904, 0.14671651019565887, 0.14671651019565896, 0.14671651019565896, 0.14671651019565896, 0.14671651019565887, 0.14671651019565898, 0.14671651019565898, 0.1467165101956589, 0.14671

M1 mechanism shape: (2, 1152)
M0 mechanism shape: (3, 2880)
Alpha_s shape: (1152, 2880)
Alpha_t shape: (2, 3)
All JS distances: [0.29800027268812596, 0.29800027268812607, 0.29800027268812584, 0.29800027268812607, 0.29800027268812596, 0.29800027268812584, 0.29800027268812607, 0.29800027268812607, 0.29800027268812584, 0.29800027268812607, 0.29800027268812607, 0.29800027268812596, 0.29800027268812596, 0.29800027268812607, 0.29800027268812584, 0.29800027268812596, 0.29800027268812596, 0.29800027268812607, 0.29800027268812584, 0.2980002726881261, 0.29800027268812596, 0.29800027268812584, 0.29800027268812596, 0.29800027268812584, 0.29800027268812607, 0.29800027268812607, 0.29800027268812607, 0.29800027268812584, 0.29800027268812607, 0.29800027268812607, 0.20700466096979914, 0.2070046609697992, 0.20700466096979894, 0.20700466096979894, 0.20700466096979914, 0.207004660969799, 0.20700466096979933, 0.20700466096979894, 0.2070046609697991, 0.2070046609697992, 0.20700466096979886, 0.20700466096979

All JS distances: [0.1458332091527631, 0.1458332091527631, 0.145833209152763, 0.14583320915276313, 0.14583320915276307, 0.145833209152763, 0.14583320915276315, 0.1458332091527631, 0.14583320915276315, 0.1458332091527631, 0.14583320915276313, 0.14583320915276315, 0.14583320915276304, 0.14583320915276313, 0.14583320915276313, 0.14583320915276318, 0.14583320915276304, 0.1458332091527631, 0.1458332091527631, 0.14583320915276307, 0.1458332091527632, 0.14583320915276302, 0.14583320915276315, 0.14583320915276315, 0.1458332091527631, 0.14583320915276313, 0.14583320915276315, 0.14583320915276302, 0.14583320915276318, 0.14583320915276313, 0.14583320915276307, 0.14583320915276307, 0.14583320915276318, 0.14583320915276313, 0.1458332091527631, 0.14583320915276307, 0.1458332091527631, 0.14583320915276313, 0.14583320915276304, 0.1458332091527631, 0.14583320915276313, 0.14583320915276307, 0.1458332091527631, 0.1458332091527631, 0.14583320915276313, 0.14583320915276313, 0.1458332091527631, 0.1458332091

Alpha_s shape: (3456, 8640)
Alpha_t shape: (2, 3)
All JS distances: [0.29800027268812607, 0.29800027268812607, 0.29800027268812596, 0.29800027268812596, 0.29800027268812607, 0.29800027268812596, 0.29800027268812607, 0.29800027268812607, 0.29800027268812596, 0.29800027268812607, 0.29800027268812607, 0.29800027268812607, 0.29800027268812596, 0.29800027268812596, 0.29800027268812607, 0.298000272688126, 0.29800027268812607, 0.29800027268812607, 0.2980002726881261, 0.29800027268812607, 0.29800027268812607, 0.29800027268812607, 0.29800027268812596, 0.298000272688126, 0.2980002726881261, 0.29800027268812607, 0.29800027268812596, 0.2980002726881261, 0.29800027268812596, 0.29800027268812596, 0.20700466096979916, 0.20700466096979928, 0.20700466096979916, 0.2070046609697992, 0.20700466096979933, 0.20700466096979916, 0.20700466096979933, 0.20700466096979928, 0.20700466096979894, 0.20700466096979891, 0.2070046609697988, 0.2070046609697991, 0.20700466096979894, 0.20700466096979916, 0.207004660969799

M0 mechanism shape: (24, 1080)
Alpha_s shape: (288, 1080)
Alpha_t shape: (24, 24)
All JS distances: [0.14750258547137535, 0.14750258547137546, 0.14750258547137532, 0.14750258547137543, 0.14750258547137526, 0.1475025854713753, 0.14750258547137537, 0.1475025854713753, 0.14750258547137543, 0.14750258547137535, 0.14750258547137535, 0.14750258547137546, 0.14750258547137537, 0.1475025854713753, 0.14750258547137543, 0.14750258547137537, 0.14750258547137532, 0.14750258547137524, 0.1475025854713754, 0.14750258547137535, 0.1475025854713753, 0.14750258547137537, 0.14750258547137543, 0.14750258547137535, 0.14750258547137537, 0.1475025854713753, 0.14750258547137535, 0.14750258547137535, 0.14750258547137532, 0.14750258547137535, 0.14750258547137532, 0.14750258547137543, 0.14750258547137543, 0.14750258547137535, 0.1475025854713754, 0.14750258547137532, 0.1475025854713753, 0.14750258547137535, 0.14750258547137535, 0.14750258547137543, 0.14750258547137543, 0.14750258547137535, 0.1475025854713753, 0.147

# Conclusion

In this notebook we have automated the procedure for computing the abstraction error between two models related by an abstraction. We have built on a series of examples, each time considering richer evaluation set over which to compute the abstraction error. The final implementation consider all possible disjoint sets of variables in the high-level model and evaluates the abstraction error over them. Meaningful and efficient computation of the abstraction error will depend on the selection of a representative and efficient evaluation set and a meaningful and robust distance metric.

## Bibliography

[Rischel2020] Rischel, Eigil Fjeldgren. "The Category Theory of Causal Models." (2020).

[Rubenstein2017] Rubenstein, Paul K., et al. "Causal consistency of structural equation models." arXiv preprint arXiv:1707.00819 (2017).

[Pearl2009] Pearl, Judea. Causality. Cambridge university press, 2009.

[Peters2017] Peters, Jonas, Dominik Janzing, and Bernhard Schölkopf. Elements of causal inference: foundations and learning algorithms. The MIT Press, 2017.

[Spivak2014] Spivak, David I. Category theory for the sciences. MIT Press, 2014.

[Fong2018] Fong, Brendan, and David I. Spivak. "Seven sketches in compositionality: An invitation to applied category theory." arXiv preprint arXiv:1803.05316 (2018).

(**TODO:** (i) select an evaluation set $\mathcal{J}$ in a smarter way; (ii) compute distances more efficiently.)